In [4]:
# Imports Train
PATH = './tweet_model'
from tweet_ner_data_label import train_data

In [5]:
# Imports & Dependencies
from __future__ import unicode_literals, print_function

import plac
import random
import warnings
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

In [ ]:
# Update original data to include Normal NER Tags to prevent forgetting problem


In [6]:
# Load model if exists
def load_model(model=None):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    
    return nlp

# Train update Spacy's NER
def train_NER(train_data, iterations, model=None):
    TRAIN_DATA = train_data
    
    # Set language to only Text in tweets
    nlp = load_model(model)
    
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pi
        
    # add new labels eg: using VENDOR_LOCATION
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    
    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    
    # only train NER
    with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():
        # show warnings for misaligned entity spans once
        warnings.filterwarnings("once", category=UserWarning, module='spacy')

        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(iterations):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)
            
    return nlp

In [7]:
# Train Data
nlp = train_NER(train_data, 1)

Created blank 'en' model
Losses {'ner': 130.8053908538856}


In [10]:
# Review NER Results
def show_ents(doc):
    doc = nlp(doc[0])
    if doc.ents:
        for ent in doc.ents:
            print(ent.text + ' - ' + ent.label_)
    else:
        print('No entities found')

In [11]:
# Test across subset of tweets
for i in range(250,275):
    show_ents(train_data[i])
    print("")

China - TRUCK_LOCATION
town - TRUCK_LOCATION
7th &amp; G st - TRUCK_LOCATION

Research Laboratory:4555 Overlook Ave SW - TRUCK_LOCATION

No entities found

No entities found

No entities found

No entities found

Union Station - TRUCK_LOCATION

union station.#2 - TRUCK_LOCATION
l'enfant plaza.#3 - TRUCK_LOCATION
metro center - TRUCK_LOCATION

Choripan - TRUCK_LOCATION

2nd &amp; M St. SE - TRUCK_LOCATION

11th and F - TRUCK_LOCATION

12 and H - TRUCK_LOCATION

STATE DEPARTMENT - TRUCK_LOCATION
CHINATOWN - TRUCK_LOCATION
METRO CENTER - TRUCK_LOCATION
BALLSTON ARLINGTON - TRUCK_LOCATION

No entities found

No entities found

Friendship Heights - TRUCK_LOCATION

waterfront metro - TRUCK_LOCATION
station - TRUCK_LOCATION

Noma - TRUCK_LOCATION

No entities found

No entities found

No entities found

No entities found

No entities found

Tysons Galleria - TRUCK_LOCATION

Union Station - TRUCK_LOCATION



In [14]:
# Manually test the trained model
test_text = "Today we are at Farragut Square"
doc = nlp(test_text)
print("Entities in '%s'" % test_text)
for ent in doc.ents:
    print(ent.label_, ent.text)

Entities in 'Today we are at Farragut Square'
TRUCK_LOCATION Farragut Square


In [ ]:
# Save model to output directory
def save_model(nlp, output_dir, name):
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta[name] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        # Check the classes have loaded back consistently
        assert nlp2.get_pipe("ner").move_names == move_names
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)

In [ ]:
save_model(nlp, './tweet_model','ner_1')